# Required: Active Bloomberg Connection

### Additionally, this notebook uses a custom Bloomberg API wrapper imported as bbapi
Using the standard Bloomberg API directly will require some modifications.

In [1]:
import numpy as np
import pandas as pd
import arch

import statsmodels.api as sm

import os, sys
import pytz
from datetime import datetime, timedelta

from sqlalchemy import create_engine

In [2]:
import sys
sys.path.insert(0, '../cmds')
from bbapi import *

In [3]:
file_out = 'proshares_analysis_data.xlsx'

ticks_hedge = [
    'HFRIFWI Index',
    'MLEIFCTR Index',
    'MLEIFCTX Index',
    'HDG US Equity',
    'QAI US Equity',
]

ticks_merrill = [
    'SPY US Equity',
    'USGG3M Index',
    'EEM US Equity',
    'EFA US Equity',
    'EUO US Equity',
    'IWM US Equity'
]

ticks_explore = [
    'TRVCI Index',
    'HEFA US Equity',
    'TAIL US Equity',
    'SPXU US Equity',
    'UPRO US Equity',
]

ticks = list(set(ticks_hedge + ticks_merrill + ticks_explore))

ticks_secs = []
ticks_indexes = []
for tick in ticks:
    if 'Equity' in tick:
        ticks_secs.append(tick)
    if 'Index' in tick:
        ticks_indexes.append(tick)

ticks_yields = 'USGG3M Index'

In [4]:
names = bb_ref(ticks, 'Name')
info = pd.DataFrame(columns=['Descriptions'])
for item in names:
    info.loc[item['ticker']] = item['value']

In [5]:
fld_secs = 'TOT_RETURN_INDEX_GROSS_DVDS'
fld_indexes = 'PX_LAST'

date_start = '20110701'
date_end = '20210930'

ANNUALIZATION = 12

In [6]:
securities = bb_hist(ticks_secs,fld_secs,date_start,date_end)
securities = securities.pivot(index='date',columns='ticker',values='value')

indexes = bb_hist(ticks_indexes,fld_indexes,date_start,date_end)
indexes = indexes.pivot(index='date',columns='ticker',values='value')

In [7]:
data_daily = securities.join(indexes,how='outer')
data_monthly = data_daily.ffill().resample('m').last()
rets = data_monthly.pct_change()
rets[ticks_yields] = data_monthly[ticks_yields]/100/ANNUALIZATION
rets = rets.iloc[1:]
rets.index = rets.index.tz_localize(None)

In [8]:
with pd.ExcelWriter(file_out) as writer:
    info.to_excel(writer, sheet_name= 'descriptions')
    rets[ticks_hedge].to_excel(writer, sheet_name='hedge_fund_series')
    rets[ticks_merrill].to_excel(writer, sheet_name='merrill_factors')
    rets[ticks_explore].to_excel(writer, sheet_name='other_data')